Attempt 07
Summarization then default approach with adaboost classifier.
Summarization created with long-t5 model.:

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.7 MB/s 
     |████████████████████████████████| 365 kB 42.2 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 6.6 MB 14.9 MB/s 
     |████████████████████████████████| 115 kB 46.7 MB/s 
     |████████████████████████████████| 141 kB 46.1 MB/s 
     |████████████████████████████████| 212 kB 6.8 MB/s 
     |████████████████████████████████| 127 kB 18.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#adapted from https://github.com/FakeNewsChallenge/fnc-1-baseline/tree/master/utils
from csv import DictReader
import random
import os
from collections import defaultdict
import sys
import os
import re
import argparse


#Loading dataset from summaries
class DataSet():
    def __init__(self, name="train", path="fnc-1", is_unlabeled=False):
        self.path = path
        print("Reading dataset")
        summaries = name + "_summaries.csv"
        stances = name + "_stances"
        if is_unlabeled is True:
            stances = stances + "_unlabeled"
        stances = stances + ".csv"

        self.stances = self.read(stances)
        articlesSummaries = self.read(summaries)
        self.summaries = dict()

        # make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        # copy all bodies into a dictionary
        for summary in articlesSummaries:
            self.summaries[int(summary['Body ID'])] = summary['articleSummary']

        print("Total stances: " + str(len(self.stances)))
        print("Total summaries: " + str(len(self.summaries)))

    def read(self, filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows


def generate_hold_out_split (dataset, training = 0.8, base_dir="splits"):
    r = random.Random()
    r.seed(1489215)

    article_ids = list(dataset.articles.keys())  # get a list of article ids
    r.shuffle(article_ids)  # and shuffle that list


    training_ids = article_ids[:int(training * len(article_ids))]
    hold_out_ids = article_ids[int(training * len(article_ids)):]

    # write the split body ids out to files for future use
    with open(base_dir+ "/"+ "training_ids.txt", "w+") as f:
        f.write("\n".join([str(id) for id in training_ids]))

    with open(base_dir+ "/"+ "hold_out_ids.txt", "w+") as f:
        f.write("\n".join([str(id) for id in hold_out_ids]))



def read_ids(file,base):
    ids = []
    with open(base+"/"+file,"r") as f:
        for line in f:
           ids.append(int(line))
        return ids


def kfold_split(dataset, training = 0.8, n_folds = 10, base_dir="splits"):
    if not (os.path.exists(base_dir+ "/"+ "training_ids.txt")
            and os.path.exists(base_dir+ "/"+ "hold_out_ids.txt")):
        generate_hold_out_split(dataset,training,base_dir)

    training_ids = read_ids("training_ids.txt", base_dir)
    hold_out_ids = read_ids("hold_out_ids.txt", base_dir)

    folds = []
    for k in range(n_folds):
        folds.append(training_ids[int(k*len(training_ids)/n_folds):int((k+1)*len(training_ids)/n_folds)])

    return folds,hold_out_ids


def get_stances_for_folds(dataset,folds,hold_out):
    stances_folds = defaultdict(list)
    stances_hold_out = []
    for stance in dataset.stances:
        if stance['Body ID'] in hold_out:
            stances_hold_out.append(stance)
        else:
            fold_id = 0
            for fold in folds:
                if stance['Body ID'] in fold:
                    stances_folds[fold_id].append(stance)
                fold_id += 1

    return stances_folds,stances_hold_out


In [3]:
# Adapted from https://github.com/FakeNewsChallenge/fnc-1/blob/master/scorer.py
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import json
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated', 'related']
RELATED = LABELS[0:3]


def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-" * line_len)
    lines.append(header)
    lines.append("-" * line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-" * line_len)
    print('\n'.join(lines))


def report_score(actual, predicted):
    score, cm = score_submission(actual, predicted)
    best_score, _ = score_submission(actual, actual)

    print_confusion_matrix(cm)
    print("Score: " + str(score) + " out of " + str(best_score) + "\t(" + str(score * 100 / best_score) + "%)")
    aprf_scores = get_precision_recall_f1_scores(actual, predicted)

    print(json.dumps(aprf_scores, indent=4, sort_keys=True, separators=(',', ': ')))
    return score * 100 / best_score

#Calculating accuracy precision recall f1 scores
def get_precision_recall_f1_scores(actual, predicted):
    # calculate precision scores for labels - Average None
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average=None)

    precisions = [{LABELS[index] : value} for (index, value) in enumerate(p)]
    recalls = [{LABELS[index] : value} for (index, value) in enumerate(r)]
    f1_scores = [{LABELS[index] : value} for (index, value) in enumerate(f1)]
    acc = accuracy_score(actual, predicted)
    avg_none = {"accuracy": acc, "precision": precisions, "recall": recalls, "f1": f1_scores}

    # calculate precision scores for labels - Average micro
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='micro')

    avg_micro = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

    # calculate precision scores for labels - Average macro
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='macro')

    avg_macro = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

    # calculate precision scores for labels - Average weighted
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='weighted')

    avg_weighted = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}


    return {"Each_Class": avg_none, "micro": avg_micro, "macro": avg_macro, "weighted": avg_weighted}

In [4]:
#adapted from https://github.com/FakeNewsChallenge/fnc-1-baseline/blob/master/feature_engineering.py
import os
import re
import nltk
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

_wnl = nltk.WordNetLemmatizer()


def normalize_word(w):
    return _wnl.lemmatize(w).lower()


def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]


def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric

    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()


def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]


def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)




def word_overlap_features(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        clean_headline = get_tokenized_lemmas(clean_headline)
        clean_body = get_tokenized_lemmas(clean_body)
        features = [
            len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))]
        X.append(features)
    return X


def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_headline = get_tokenized_lemmas(clean_headline)
        features = [1 if word in clean_headline else 0 for word in _refuting_words]
        X.append(features)
    return X


def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        features = []
        features.append(calculate_polarity(clean_headline))
        features.append(calculate_polarity(clean_body))
        X.append(features)
    return np.array(X)


def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features


def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features


def hand_features(headlines, bodies):

    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph

        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        X.append(binary_co_occurence(headline, body)
                 + binary_co_occurence_stops(headline, body)
                 + count_grams(headline, body))


    return X


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
#adapted from https://github.com/FakeNewsChallenge/fnc-1-baseline/blob/master/fnc_kfold.py
import sys
import numpy as np


def generate_features(stances, dataset, name):
    h, b, y = [], [], []

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.summaries[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap." + name + ".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting." + name + ".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity." + name + ".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "features/hand." + name + ".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X, y


def generate_features_without_labels(stances, dataset, name):
    h, b, y = [], [], []

    for stance in stances:
        h.append(stance['Headline'])
        b.append(dataset.summaries[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap." + name + ".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting." + name + ".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity." + name + ".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "features/hand." + name + ".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X

In [6]:
import sys
import numpy as np

from sklearn.ensemble import AdaBoostClassifier
import pandas as pd

#check_version()
#parse_params()

# Load the training dataset and generate folds
d = DataSet()
folds, hold_out = kfold_split(d, n_folds=10)
fold_stances, hold_out_stances = get_stances_for_folds(d, folds, hold_out)

# Load the competition dataset
unlabeled_competition_dataset = DataSet(name="competition_test", is_unlabeled=True)
X_unlabeled = generate_features_without_labels(unlabeled_competition_dataset.stances, unlabeled_competition_dataset,"competition_unlabeled")

Xs = {}
ys = {}

# Load/Precompute all features now
X_holdout, y_holdout = generate_features(hold_out_stances, d, "holdout")
for fold in fold_stances:
    Xs[fold], ys[fold] = generate_features(fold_stances[fold], d, str(fold))

best_score = 0
best_fold = None

# Classifier for each fold
for fold in fold_stances:
    ids = list(range(len(folds)))
    del ids[fold]

    X_train = np.vstack(tuple([Xs[i] for i in ids]))
    y_train = np.hstack(tuple([ys[i] for i in ids]))

    X_test = Xs[fold]
    y_test = ys[fold]

    clf = AdaBoostClassifier(n_estimators=200, random_state=4563)
    clf.fit(X_train, y_train)

    predicted = [LABELS[int(a)] for a in clf.predict(X_test)]
    actual = [LABELS[int(a)] for a in y_test]

    fold_score, _ = score_submission(actual, predicted)
    max_fold_score, _ = score_submission(actual, actual)

    score = fold_score / max_fold_score

    print("Score for fold " + str(fold) + " was - " + str(score))
    if score > best_score:
        best_score = score
        best_fold = clf

# Run on Holdout set and report the final score on the holdout set
predicted = [LABELS[int(a)] for a in best_fold.predict(X_holdout)]
actual = [LABELS[int(a)] for a in y_holdout]

print("Scores on the dev set")
report_score(actual, predicted)
print("")
print("")

# Run on competition dataset
predicted = [LABELS[int(a)] for a in best_fold.predict(X_unlabeled)]
print(predicted)

result = []
for i, predictedLabel in enumerate(predicted):
    dict = {
        "Headline": unlabeled_competition_dataset.stances[i]['Headline'],
        "Body ID": unlabeled_competition_dataset.stances[i]['Body ID'],
        "Stance": predictedLabel
    }
    result.append(dict)

test_data = pd.DataFrame(result)
test_data.to_csv('answer.csv', index=False, encoding='utf-8')  # From pandas library


Reading dataset
Total stances: 49972
Total summaries: 1683
Reading dataset
Total stances: 25413
Total summaries: 904


25413it [00:34, 737.39it/s]
25413it [00:06, 4007.73it/s]
25413it [00:24, 1027.66it/s]
25413it [00:25, 1015.98it/s]
9622it [00:09, 1064.31it/s]
9622it [00:02, 4433.34it/s]
9622it [00:08, 1098.79it/s]
9622it [00:09, 1058.76it/s]
4124it [00:03, 1110.30it/s]
4124it [00:00, 4315.42it/s]
4124it [00:04, 973.15it/s] 
4124it [00:04, 1012.81it/s]
4663it [00:04, 1070.50it/s]
4663it [00:01, 4322.58it/s]
4663it [00:04, 936.59it/s] 
4663it [00:04, 965.73it/s]
3783it [00:03, 1120.59it/s]
3783it [00:00, 4647.64it/s]
3783it [00:03, 1157.93it/s]
3783it [00:03, 1118.43it/s]
3388it [00:03, 1031.15it/s]
3388it [00:00, 4607.51it/s]
3388it [00:03, 1030.42it/s]
3388it [00:03, 1017.17it/s]
3644it [00:03, 1079.37it/s]
3644it [00:00, 4680.83it/s]
3644it [00:03, 1038.18it/s]
3644it [00:03, 985.65it/s] 
4644it [00:04, 959.82it/s]
4644it [00:01, 3773.23it/s]
4644it [00:04, 971.15it/s]
4644it [00:04, 1014.72it/s]
3848it [00:03, 1055.62it/s]
3848it [00:00, 4114.90it/s]
3848it [00:03, 969.32it/s]
3848it [00:03, 985.77

Score for fold 6 was - 0.7223498994541798
Score for fold 0 was - 0.7210100514832066
Score for fold 7 was - 0.7452976948097865
Score for fold 5 was - 0.7075110456553756
Score for fold 2 was - 0.6994765920325676
Score for fold 8 was - 0.7554285714285714
Score for fold 9 was - 0.7102986324309238
Score for fold 3 was - 0.7722304682229119
Score for fold 1 was - 0.7137868417376728
Score for fold 4 was - 0.7320074402632708
Scores on the dev set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    75     |     4     |    525    |    158    |
-------------------------------------------------------------
| disagree  |    16     |     0     |    120    |    26     |
-------------------------------------------------------------
|  discuss  |    54     |     1     |   1317    |    428    |
-------------------------------------------------------------


Received 8124.0 points on leatherboard. Summarization resulted in worse scores than default approach.